In [11]:
from config import config
from qm.qua import program,for_,stream_processing,declare,declare_stream,wait,measure,play,save,fixed,demod,amp,ramp, align
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm import SimulationConfig
import time as time
import matplotlib.pyplot as plt
import matplotlib.animation as ani
import numpy as np


qmm = QuantumMachinesManager(host='192.168.15.128',port=80)
qmm.close_all_quantum_machines()
qm = qmm.open_qm(config)




with program() as sprog:
    var=declare(fixed,value=np.linspace(-0.1,0.3,5))
    ind=declare(int)
    I=declare(fixed)
    Q=declare(fixed)
    with for_(ind,0,ind<5,ind+1):
        # align('Q1_readout','Q1_L')
        
        play('CW'*amp(0.1), 'Q1_L', duration=100//4)
        measure('readout_pulse_0_05', 'Q2_readout', None, demod.full('cos', I , 'out1'), demod.full('sin', Q , 'out1'))
        # play('CW'*amp(-var[ind]), 'Q1_L', duration=100//4)
    


job = qm.simulate(sprog, SimulationConfig(int(3000//4)))
samples = job.get_simulated_samples()
samples.con1.plot()


2022-04-20 17:30:27,609 - qm - INFO - Performing health check
2022-04-20 17:30:27,612 - qm - INFO - Health check passed
2022-04-20 17:30:27,770 - qm - INFO - to simulate a program, use QuantumMachinesManager.simulate(..)
2022-04-20 17:30:27,815 - qm - INFO - Flags: 
2022-04-20 17:30:27,816 - qm - INFO - Simulating program
Error in callback <function flush_figures at 0x000002984AC58B80> (for post_execute):


KeyboardInterrupt: 

# just testing the pulser part

In [1]:
def hold_action(channel,time,looped=False):
    return {'action':'hold' , 'channel':channel , 'action_variables':{'time':time},'looped':looped}

def step_action(channel,value,time,looped=False,looped_variable='step_value',loop_index=0):
    return {'action':'step' , 'channel':channel , 'action_variables':{'time':time,'step_value':value},'looped':looped,'looper':looped_variable,'loop_index':loop_index}

def ramp_action(channel,rate,time,looped=False):
    return {'action':'ramp' , 'channel':channel , 'action_variables':{'time':time,'rate':rate},'looped':looped}

def meas_action(channel,type='full',pulse='readout_pulse_0_05',looped=False,buffer_size=16):
    return {'action':'meas' , 'channel':channel , 'action_variables':{'type':type , 'pulse':pulse,'buffer_size':buffer_size},'looped':looped}


from qmachine.pulse_generator.config import config
from qmachine.pulse_generator.pulse_generator import Pulser
import numpy as np
pulse_maker=Pulser(config)

actions_m1={'steps':{
                '1':
                    {'ch1':step_action(channel='ch1',value=np.linspace(0.1,0.3,4),time=1000//4,looped=True,looped_variable='step_value',loop_index='0'),
                    'ch2':step_action('ch2',np.linspace(-0.1,-0.3,4),1000//4,True,'step_value',loop_index='1'),
                    'm1':meas_action('m1',buffer_size=[4,4])},
                '2':
                    {'ch1':step_action('ch1',np.linspace(-0.1,-0.3,4),1000//4,True,'step_value',loop_index='0'),
                    'ch2':step_action('ch2',np.linspace(0.1,0.3,4),1000//4,True,'step_value',loop_index='1')}},
        'looped':[4,4],
        'channels':['ch1','ch2','m1']}


actions_m2={'steps':{
                '1':
                    {'ch1':step_action('ch1',np.linspace(0.1,0.3,4),1000//4,True,'step_value',loop_index='0'),
                    'ch2':step_action('ch2',np.linspace(-0.1,-0.3,4),1000//4,True,'step_value',loop_index='1')},
                '2':
                    {'ch1':step_action('ch1',np.linspace(-0.1,-0.3,4),1000//4,True,'step_value',loop_index='0'),
                    'ch2':step_action('ch2',np.linspace(0.1,0.3,4),1000//4,True,'step_value',loop_index='1'),
                    'm1':meas_action('m1',buffer_size=[4,4])}},
        'looped':[4,4],
        'channels':['ch1','ch2','m1']}

# actions={'steps':{
#                 '1':
#                     {'ch1':step_action('ch1',0.1,1000//4,False,'step_value',loop_index='0')},
#                 '2':
#                     {'ch1':step_action('ch1',-0.1,1000//4,False,'step_value',loop_index='0')}},
#         'channels':['ch1']}


# import pprint
# pprint.pprint(actions)

seq,mod_actions=pulse_maker.build_seq(actions_m2)

# job=pulse_maker.simulate_pulse(seq,32000//4+300//4)
pulse_maker.open_qm(pulse_maker.config)
job=pulse_maker.qm.execute(seq)

2022-04-22 17:14:36,194 - qm - INFO - Performing health check
2022-04-22 17:14:36,197 - qm - INFO - Health check passed
quantum machine opened with channels ['Q1_R', 'Q1_L', 'Q1_readout', 'Q2_readout']
default value for CW is: 0.1
2022-04-22 17:14:36,273 - qm - INFO - Flags: 
2022-04-22 17:14:36,274 - qm - INFO - Sending program to QOP
2022-04-22 17:14:36,388 - qm - INFO - Executing program


In [2]:
import pprint
pprint.pprint(mod_actions['steps']['2']['m1'])

{'action': 'meas',
 'action_variables': {'I_name': 'I_1',
                      'Q_name': 'Q_1',
                      'buffer_size': 16,
                      'pulse': 'readout_pulse_0_05',
                      'save_I': <qm.qua._dsl._Expression object at 0x0000018531559930>,
                      'save_I_stream': <qm.qua._dsl._ResultSource object at 0x0000018531559DB0>,
                      'save_Q': <qm.qua._dsl._Expression object at 0x0000018531559CC0>,
                      'save_Q_stream': <qm.qua._dsl._ResultSource object at 0x0000018531559E70>,
                      'type': 'full'},
 'channel': 'm1',
 'looped': False}


In [2]:
all_results= {key : item.fetch_all() for key,item in job.result_handles._all_results.items()}

In [3]:
all_results

{'I_1': array([-5.04739583e-05, -5.22769988e-05, -3.22274864e-05, -6.21303916e-05,
        -2.14204192e-05,  1.92224979e-05,  2.99401581e-05, -6.83218241e-06,
         1.23083591e-05,  8.04662704e-07,  2.08579004e-05,  2.51829624e-05,
         5.02914190e-05, -2.60174274e-05, -2.62148678e-05,  3.11844051e-05]),
 'Q_1': array([ 3.20747495e-05,  3.58261168e-05,  7.55116343e-05, -2.22027302e-06,
        -2.25640833e-05,  4.00096178e-06, -7.14138150e-05, -1.71884894e-05,
        -4.48897481e-05, -1.02445483e-06,  3.63327563e-05,  2.24262476e-06,
         4.25390899e-05, -2.29775906e-05,  2.64495611e-07, -6.66230917e-05])}

In [4]:
t=job.result_handles._all_results

In [6]:
t['I_1'].fetch_all() 

array([ 3.88547778e-06, -1.62497163e-05,  4.12203372e-05, -9.62987542e-06,
        2.65985727e-06,  9.04127955e-06, -1.67600811e-05, -4.88013029e-07,
        7.64429569e-06,  1.03887171e-04, -1.86823308e-05, -1.85184181e-05,
       -1.63838267e-05, -1.90995634e-05,  6.74873590e-05, -2.26870179e-05])